<a href="https://colab.research.google.com/github/kimhwijin/TensorflowWithKeras/blob/master/RNN/seq2seq_translate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import nltk
import numpy as np
import re
import shutil
import tensorflow as tf
import os
import unicodedata
import zipfile
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction


In [3]:
def preprocessing_sentence(sent):
    sent = "".join([c for c in unicodedata.normalize("NFD", sent) if unicodedata.category(c) != "Mn"])
    #문자열 앞에 r 이붙으면 그대로 반환 r'abcd\n' = abcd\n
    sent = re.sub(r"([!.?])", r" \1", sent)
    #알파벳 또는 ! ? 제외하고 공백으로 치환
    sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)
    #공백문자를 띄어쓰기 한칸으로 변경
    sent = re.sub(r"\s+", " ", sent)
    sent = sent.lower()
    return sent

def download_and_read(url, num_sent_pairs=30000):

    local_file = url.split('/')[-1]
    drive_path = "drive/MyDrive/Datasets/anki-fra-eng"
    data_path = os.join.path(drive_file, local_file)
    if not os.path.exists(data_path):
        os.system('-wget -O {:s} {:s}'.format(local_file, url))
        with zipfile.ZipFile(data_path, 'r') as zip_ref:
            zip_ref.extractall(data_path)
    file_path = os.join.path(data_path, 'fra_txt')
    en_sents, fr_sents, fr_sents_out = [], [], []

    with open(file_path, 'r') as fin:
        for i , line in enumerate(fin):
            en_sent, fr_sent = line.strip().split('\t')
            en_sent = [w for w in preprocessing_sentence(en_sent).split()]
            fr_sent = preprocessing_sentence(fr_sent)
            fr_sent_in = [w for w in ("BOS" + fr_sent).split()]
            fr_sent_out = [w for w in (fr_sent + "EOS").split()]
            en_sents.append(en_sent)
            fr_sents_in.append(fr_sent_in)
            fr_sents_out.append(fr_sent_out)
            if i >= num_sent_pairs - 1:
                break
    return en_sents, fr_sents_in, fr_sents_out


In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, num_timestemps, embedding_dim, encoder_dim, **kwargs):
        super(Encoder, self).__init__(**kwargs)
        self.encoder_dim = encoder_dim
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=timestemps)
        self.rnn = tf.keras.layers.GRU(encoder_dim, return_sequences=True, return_state=True)

    def call(self, x, state):
        x = self.embedding(x)
        x, state = self.rnn(x, initial_state=state)
        return x, state
    def init_state(self, batch_size):
        return tf.zeros((batch_size, encoder_dim))

class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, num_timestemps, embedding_dim, decoder_dim, **kwargs):
        super(Decoder, self).__init__(**kwargs)
        self.decoder_dim = decoder_dim
        self.embedding = tf.keras.layers.Embedding(vocab_size, decoder_dim, input_length=num_timestemps)
        self.rnn = tf.keras.layers.GRU(decoder_dim, return_sequences=True, return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)
    
    def call(self, x, state):
        x = self.embedding(x)
        x, state = self.rnn(x, state)
        x = self.dense(x)
        return x, state

In [ ]:
#하이퍼 파라미터
NUM_SENT_PAIRS = 30000
EMBEDDING_DIM = 256
ENCODER_DIM, DECODER_DIM = 1024, 1024
BATCH_SIZE = 64
NUM_EPOCHS = 30

#문장 데이터
sents_en, sents_fr_in, sents_fr_out = download_and_read('https://www.manythings.org/anki/fra-eng.zip', NUM_SENT_PAIRS)

In [ ]:
#토크나이저
tokenizer_en = tf.keras.preprocessing.text.Tokenizer(filters="", lower=False)
tokenizer_en.fit_on_texts(sents_en)
data_en = tokenizer_en.texts_to_sequences(sents_en)
#뒤쪽 빈 부분을 채워줌
data_en = tf.keras.preprocessing.sequence.pad_sequences(data_en, padding='post')

#데이터 및 토크나이저 설정
tokenizer_fr = tf.keras.preprocessing.text.Tokenizer(filters="", lower=False)
tokenizer_fr.fit_on_texts(sents_fr_in)
tokenizer_fr.fit_on_texts(sents_fr_out)
data_fr_in = tokenizer_fr.texts_to_sequences(sents_fr_in)
data_fr_out = tokenizer_fr.texts_to_sequences(sents_fr_out)
data_fr_in = tf.keras.preprocessing.sequence.pad_sequences(data_fr_in, padding='post')
data_fr_out = tf.keras.preprocessing.sequence.pad_sequences(data_fr_out, padding='post')

#단어 개수
vocab_size_en = len(tokenizer_en.word_index)
vocab_size_fr = len(tokenizer_fr.word_index)
word2idx_en = tokenizer_en.word_index
idx2word_en = {v: k for k , v in word2idx_en.items()}
word2idx_fr = tokenizer_fr.word_index
idx2word_fr = {v: k for k , v in word2idx_fr.items()}
print("단어 사이즈 (en) : {:d}, (fr) : {:d}".format(vocab_size_en, vocab_size_fr))
maxlen_en = data_en.shape[1]
maxlen_fr = data_fr_out.shape[1]
print("기준 시퀀셜 길이 (en) : {:d}, (fr) : {:d}".format(maxlen_en, maxlen_fr))

In [ ]:
#test, train dataset // 1 : 3 비율
dataset = tf.data.Dataset.from_tensor_slices((data_en, data_fr_in, data_fr_out))
dataset = dataset.shuffle(10000)
test_size = NUM_SENT_PAIRS // 4
test_dataset = dataset.take(test_size).batch(batch_size, drop_remainder=True)
train_dataset = dataset.skip(test_size).batch(batch_size, drop_remainder=True)